In [1]:
# #20가지 주제의 뉴스 데이터
# import pandas as pd
# from sklearn.datasets import fetch_20newsgroups
# dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                              remove=('headers', 'footers', 'quotes'))
# documents = dataset.data
# len(documents)

11314

In [2]:
# documents[0] #첫번째 뉴스

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [3]:
# #뉴스 카테고리
# print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [1]:
import pandas as pd
f = open("c:/data/text/news2.txt")
lines = f.readlines()
f.close()

In [2]:
news_df = pd.DataFrame({'document':lines})
# 알파벳 이외의 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

# 길이가 3이하인 단어 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

# 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
news_df['clean_doc'][0]

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_11664\183726181.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


'recent days boss russian private military company wagner seems have gone into social media meltdown flooding telegram channel other accounts with ever more outrageous provocative statements'

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# 토큰화
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

# 불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
print(tokenized_doc[0])

['recent', 'days', 'boss', 'russian', 'private', 'military', 'company', 'wagner', 'seems', 'gone', 'social', 'media', 'meltdown', 'flooding', 'telegram', 'channel', 'accounts', 'ever', 'outrageous', 'provocative', 'statements']


In [4]:
# tf-idf 행렬을 만들기 위해 다시 역토큰화
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
news_df['clean_doc'] = detokenized_doc
news_df['clean_doc'][0]

'recent days boss russian private military company wagner seems gone social media meltdown flooding telegram channel accounts ever outrageous provocative statements'

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
#상위 1000개의 단어만 처리
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(42, 329)

In [6]:
from sklearn.decomposition import TruncatedSVD
# 행렬 특이값 분해, 11314개의 행을 20개로 축소, n_components 토픽수
svd_model = TruncatedSVD(n_components=20)
svd_model.fit(X)
len(svd_model.components_)

20

In [7]:
import numpy as np
#토픽수 x 단어수
np.shape(svd_model.components_)

(20, 329)

In [8]:
svd_model.components_

array([[ 0.00986489,  0.00986489,  0.0544601 , ...,  0.00986489,
         0.00986489,  0.01699437],
       [ 0.00789978,  0.00789978, -0.03864047, ...,  0.00789978,
         0.00789978, -0.00108374],
       [ 0.02477931,  0.02477931, -0.03909673, ...,  0.02477931,
         0.02477931,  0.01170998],
       ...,
       [-0.00840577, -0.00840577, -0.07185177, ..., -0.00840577,
        -0.00840577,  0.02290138],
       [ 0.00922385,  0.00922385, -0.02704128, ...,  0.00922385,
         0.00922385, -0.01892924],
       [ 0.01470905,  0.01470905,  0.0249326 , ...,  0.01470905,
         0.01470905, -0.03592365]])

In [9]:
# 단어 집합, 1000개의 단어
terms = vectorizer.get_feature_names_out()
#20개의 뉴스그룹별로 추출한 토픽 리스트 출력
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1),
            [(feature_names[i], 
                topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)
#각 토픽의 핵심 키워드 추출
#LSA: 쉽고 빠르게 구현이 가능하지만 새로운 데이터가 추가되면 처음부터 다시 계산을 해야 하는 단점이 있음

Topic 1: [('prigozhin', 0.29893), ('military', 0.28873), ('russia', 0.22883), ('putin', 0.18869), ('russian', 0.18743)]
Topic 2: [('putin', 0.43483), ('politically', 0.2426), ('expendable', 0.2426), ('prigozhin', 0.15352), ('kremlin', 0.13493)]
Topic 3: [('shells', 0.39404), ('save', 0.15476), ('warehouses', 0.15284), ('receive', 0.13317), ('grandpa', 0.13317)]
Topic 4: [('military', 0.23153), ('public', 0.21814), ('leadership', 0.19707), ('criticism', 0.1641), ('continued', 0.15931)]
Topic 5: [('expendable', 0.46448), ('politically', 0.46448), ('star', 0.20358), ('political', 0.19516), ('shells', 0.11824)]
Topic 6: [('star', 0.33274), ('political', 0.30897), ('putin', 0.16767), ('social', 0.08527), ('media', 0.08527)]
Topic 7: [('star', 0.32048), ('victory', 0.2912), ('political', 0.29098), ('flanks', 0.26678), ('today', 0.18174)]
Topic 8: [('exactly', 0.2349), ('political', 0.19349), ('driving', 0.17819), ('simply', 0.17819), ('tether', 0.17819)]
Topic 9: [('observers', 0.29281), ('o